<a href="https://colab.research.google.com/github/yeonmi02/CPR_project/blob/main/Imqur_Chill_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import time
import pandas as pd
from tqdm import tqdm

# ✅ Imgur API 설정
CLIENT_ID = "9ee03e0f723d1fc"  # 여기에 본인의 클라이언트 ID 입력
HEADERS = {"Authorization": f"Client-ID {CLIENT_ID}"}

# ✅ 검색 태그 및 설정
TAGS = ["memes", "funny", "dankmemes"]
MAX_IMAGES = 1500
PER_PAGE = 60  # 한 번에 가져올 수 있는 최대 수
collected_data = []

# ✅ 수집 루프
for tag in TAGS:
    page = 0
    total_collected = 0
    print(f"\n▶ 태그 '{tag}'에서 이미지 수집 중...")

    while total_collected < MAX_IMAGES:
        url = f"https://api.imgur.com/3/gallery/t/{tag}/time/all/{page}"
        res = requests.get(url, headers=HEADERS)

        if res.status_code != 200:
            print(f"❌ 오류: {res.status_code}")
            break

        data = res.json()
        items = data.get("data", {}).get("items", [])
        if not items:
            print("⚠️ 더 이상 이미지 없음.")
            break

        for item in items:
            if "images" in item:
                for img in item["images"]:
                    if img.get("type", "").startswith("image/"):
                        collected_data.append({
                            "title": item.get("title", ""),
                            "link": img.get("link", ""),
                            "tag": tag
                        })
                        total_collected += 1
                        if total_collected >= MAX_IMAGES:
                            break
        page += 1
        time.sleep(1)  # 너무 빠르면 차단될 수 있으므로 딜레이

# ✅ 결과 저장
df = pd.DataFrame(collected_data)
df.to_csv("imgur_memes.csv", index=False)
print(f"\n✅ 총 수집된 이미지 수: {len(df)}개")


▶ 태그 'memes'에서 이미지 수집 중...

▶ 태그 'funny'에서 이미지 수집 중...

▶ 태그 'dankmemes'에서 이미지 수집 중...

✅ 총 수집된 이미지 수: 4590개


In [4]:
import pandas as pd

# ✅ 1. Chill Guy 스타일 키워드 정의
chill_keywords = [
    "chill", "this is fine", "calm", "peaceful", "nothing wrong", "everything is fine",
    "cool guy", "relaxed", "on fire", "me watching", "world burns", "unbothered"
]

# ✅ 2. Imgur 데이터 불러오기
df = pd.read_csv("imgur_memes.csv")  # ← 파일명 확인

# ✅ 3. Chill Guy 여부 판단 함수
def is_chill_meme(text):
    if pd.isna(text):
        return False
    return any(keyword in text.lower() for keyword in chill_keywords)

# ✅ 4. 제목, 설명, 태그에서 필터링
df["is_chill"] = df.apply(
    lambda row: is_chill_meme(str(row.get("title", ""))) or
                is_chill_meme(str(row.get("description", ""))) or
                is_chill_meme(str(row.get("tags", ""))),
    axis=1
)

# ✅ 5. 필터링된 데이터 저장
chill_df = df[df["is_chill"]]
chill_df.to_csv("chill_guy_memes_from_imgur.csv", index=False)

print(f"🎯 필터링된 Chill Guy 밈 수: {len(chill_df)}개")

🎯 필터링된 Chill Guy 밈 수: 1개


In [6]:
import requests
import pandas as pd
import time

CLIENT_ID = "9ee03e0f723d1fc"  # 여기에 본인의 Client ID 입력
headers = {'Authorization': f'Client-ID {CLIENT_ID}'}

# ✅ 검색할 키워드 리스트
search_terms = [
    "this is fine", "chill guy", "relaxed meme", "peaceful chaos",
    "on fire meme", "calm in chaos", "everything is fine meme", "unbothered meme"
]

all_results = []

# ✅ 키워드 순회하며 Imgur에서 이미지 수집
for term in search_terms:
    print(f"🔍 검색 중: '{term}'")
    for page in range(5):  # 각 키워드 당 5페이지씩 (보통 60~100개 정도)
        url = f"https://api.imgur.com/3/gallery/search/top/all/{page}?q={term}"
        res = requests.get(url, headers=headers)
        if res.status_code != 200:
            print(f"❌ 요청 실패: {res.status_code}")
            break
        data = res.json()["data"]
        for item in data:
            if item.get("type", "").startswith("image"):
                all_results.append({
                    "id": item.get("id"),
                    "title": item.get("title"),
                    "description": item.get("description"),
                    "tags": ", ".join([tag["name"] for tag in item.get("tags", [])]),
                    "link": item.get("link"),
                    "datetime": item.get("datetime")
                })
        time.sleep(1)  # API 과속 방지용

# ✅ 데이터프레임으로 정리하고 중복 제거
df = pd.DataFrame(all_results)
df.drop_duplicates(subset="id", inplace=True)
print(f"🎯 수집된 밈 개수: {len(df)}개")

# ✅ 저장
df.to_csv("chill_guy_meme_candidates.csv", index=False)


chill_keywords = [
    "chill", "this is fine", "calm", "peaceful", "nothing wrong", "everything is fine",
    "cool guy", "relaxed", "on fire", "me watching", "world burns", "unbothered"
]

def is_chill_meme(text):
    if pd.isna(text):
        return False
    return any(keyword in text.lower() for keyword in chill_keywords)

df = pd.read_csv("chill_guy_meme_candidates.csv")

df["is_chill"] = df.apply(
    lambda row: is_chill_meme(str(row.get("title", ""))) or
                is_chill_meme(str(row.get("description", ""))) or
                is_chill_meme(str(row.get("tags", ""))),
    axis=1
)

filtered = df[df["is_chill"]]
filtered.to_csv("chill_guy_memes_final.csv", index=False)
print(f"✅ 최종 Chill Guy 밈 수: {len(filtered)}개")

🔍 검색 중: 'this is fine'
🔍 검색 중: 'chill guy'
🔍 검색 중: 'relaxed meme'
🔍 검색 중: 'peaceful chaos'
🔍 검색 중: 'on fire meme'
🔍 검색 중: 'calm in chaos'
🔍 검색 중: 'everything is fine meme'
🔍 검색 중: 'unbothered meme'
🎯 수집된 밈 개수: 98개
✅ 최종 Chill Guy 밈 수: 81개
